In [6]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

SOC_URL = "./actorMovies.csv"
actorMovies = pd.read_csv(SOC_URL, sep=";")
#Get all actors and movies in a np array
actors = actorMovies["Actor"]
movies = actorMovies["Movies"]
#Iterate over each actor and split movies
for i in range(actorMovies.count()[0]):
    movies[i] = movies[i].split("|")
  
actorMovies.head()

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,Actor,Movies
0,Karen Allen,"[Malcolm X, National Lampoon's Animal House, S..."
1,Taye Diggs,"[House on Haunted Hill, Go, Basic, Chicago, Eq..."
2,Michael Murphy,"[Salvador, Cloak & Dagger, Nashville, Salvador..."
3,Les Tremayne,"[The War of the Worlds, The War of the Worlds]"
4,Bridgette Wilson,"[Mortal Kombat, Nixon, House on Haunted Hill]"


In [2]:
total = 0
movies_set = []

for ID in range(len(actors)):
    for j in range(len(movies[ID])):
        movies_set.append(movies[ID][j])
    total += len(movies[ID])
movies_set=np.unique(movies_set) #Make a set do not contains duplicates of different movies
print("Different movies:",len(movies_set))

Different movies: 1896


In [3]:
#NetworkX Procesing
import networkx as nx
import time
GM = nx.MultiGraph() #Allows duplicate edges (1,2) and (2,1) -> add both
G = nx.Graph() #Doesn't allow duplicate edges (1,2) and (2,1) -> add last

#Added Nodes -> Films
G.add_nodes_from(movies_set[:])
GM.add_nodes_from(movies_set[:])


#Add Edges, pairs (film,film)
def add_all_edges(G):
    for i in range(len(movies)):
        for j in range(len(movies[i])-1):
        #Dont go over the full list -> quicker
        #why? -> last element connected in previous iter
            for k in range(len(movies[i])):
                if(j >= k): #Dont add equal edges or already added
                    str1 = movies[i][j]
                    str2 = movies[i][k]
                    if(str1 != str2): #Some actors has duplicated movies
                        G.add_edge(str1,str2)

start_time = time.time() #Compute time of execution
add_all_edges(G)
add_all_edges(GM)
print("--- %s seconds ---" % (time.time() - start_time))
print("Nº Edges:",G.number_of_edges())


--- 2.3520331382751465 seconds ---
Nº Edges: 12402


In [4]:
import networkx as nx
#Some analysis of the network
x = nx.degree_centrality(G)
xM = nx.degree_centrality(GM)

print("--------------------------SimpleGraph----------------------------------")
i = 0
for k,v in sorted(x.items(), key=lambda p:p[1], reverse=True):
    if(i < 20):
        print(k,v)
    i+=1

print("--------------------------MultiGraph----------------------------------")
i = 0
for k,v in sorted(xM.items(), key=lambda p:p[1], reverse=True):
    if(i < 20):
        print(k,v)
    i+=1

--------------------------SimpleGraph----------------------------------
Hamlet 0.08232189973614776
Pulp Fiction 0.04327176781002638
The Lord of the Rings: The Return of the King 0.04010554089709763
Armageddon 0.039577836411609495
Dune 0.03799472295514512
Heat 0.03799472295514512
JFK 0.03799472295514512
Nixon 0.03746701846965699
Sin City 0.03746701846965699
The Lord of the Rings: The Fellowship of the Ring 0.03746701846965699
Austin Powers: Goldmember 0.03588390501319261
Dead Men Don't Wear Plaid 0.03588390501319261
Cold Mountain 0.03430079155672823
How the West Was Won 0.033773087071240104
The Lord of the Rings: The Two Towers 0.033773087071240104
Enemy of the State 0.03324538258575198
North 0.03271767810026385
The Thin Red Line 0.031134564643799472
Titanic 0.031134564643799472
Harry Potter and the Goblet of Fire 0.030606860158311343
--------------------------MultiGraph----------------------------------
Hamlet 0.09445910290237466
Harry Potter and the Order of the Phoenix 0.073878627968

### Centralidad en diferentes tipos de grafo
Podemos obersvar que en el multigrafo al tener conexiones duplicadas
Sagas de peliculas famosas donde actuan la misma gente aumenta considerablemente su nivel de centralidad  

---
>  
- Pulp Fiction -> **2ª** posicion en grafo simple  
- Pulp Fiction -> **9ª** posicion en un multigrafo  

Y antes de todas estas nos encontramos con la saga completa de **Harry Potter**



In [5]:

def get_movie_list(actor_name):
    return actorMovies.loc[actorMovies["Actor"] == actor_name].iloc[0][1]
    
def get_movie_cast(movie_name):
    cast = []
    for i in range(actorMovies.count()[0]):
        mov_list = actorMovies.loc[i]
        if(movie_name in mov_list.iloc[1]):
            cast.append(mov_list[0])
    return cast

print("Hamlet total cast:",len(get_movie_cast("Hamlet")))
print("Pulp Fiction total cast:", len(get_movie_cast("Pulp Fiction")))
print(get_movie_cast("Pulp Fiction"))

def count_total_movies(movie_name):
    total = 0
    for i in range(len(get_movie_cast(movie_name))):
        total += len(get_movie_list(get_movie_cast(movie_name)[i]))
    return total

#Commented Out -> Ineficient function
#print(count_total_movies("Hamlet")) #304
#print(count_total_movies("Pulp Fiction")) #112


Hamlet total cast: 49
Pulp Fiction total cast: 12
['Frank Whaley', 'Uma Thurman', 'Christopher Walken', 'Steve Buscemi', 'Samuel L. Jackson', 'John Travolta', 'Eric Stoltz', 'Bruce Willis', 'Amanda Plummer', 'Harvey Keitel', 'Ving Rhames', 'Tim Roth']


### ¿Porque es mayor una que otra?

Como podemos ver Hamlet **cuadruplica** numero de actores de Pulp Fiction, por lo que es mas facil que esta pelicula este conectado con otras en mayor medida, pues tiene mas puntos de conexion con otras.

Sin embargo Pulp Fiction aun teniendo muy pocos actores(de los mas bajos entre los 10 primeros), tiene actores mas famosos que han participado en muchas peliculas

In [6]:
#Print Graph & Save
nx.write_gml(G, "graph.gml")
nx.write_gml(GM, "multi-graph.gml")